## Compiling functions with Cython

In [1]:
import pyximport

In [2]:
%load_ext Cython

Function: $y = (x^2 + 3) / 4.5$

In [3]:
def my_op(data):
    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [4]:
%%cython

def my_cop(data):
    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [5]:
my_op

<function __main__.my_op(data)>

In [6]:
my_cop

<function _cython_magic_1b58b035c9f60d962b9d8e5ba1563dcd.my_cop>

In [7]:
%timeit my_op([1,2,3,4,5])

2.13 µs ± 7.23 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [8]:
%timeit my_cop([1,2,3,4,5])

1.21 µs ± 112 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [9]:
import numpy

In [10]:
numpy.array([1,2,3,4,5])

array([1, 2, 3, 4, 5])

In [11]:
d = numpy.array([1,2,3,4,5])
( d ** 2 + 3 ) / 4.5

array([0.88888889, 1.55555556, 2.66666667, 4.22222222, 6.22222222])

In [12]:
%timeit ( d ** 2 + 3 ) / 4.5

3.02 µs ± 95 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [13]:
bigdata = numpy.random.randn(1000000)
bigdata.dtype

dtype('float64')

In [14]:
%timeit my_op(bigdata.copy())

836 ms ± 72.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit ( bigdata.copy() ** 2 + 3 ) / 4.5

2.51 ms ± 10 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%timeit my_cop(bigdata.copy())

772 ms ± 54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%cython
cimport numpy as np

def my_cop(np.ndarray[np.float_t, ndim=1] data):
    cdef int i

    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [18]:
my_cop

<function _cython_magic_665fa0374f771fcc24895eadf7f4b9f1.my_cop>

In [19]:
%timeit my_cop(bigdata.copy())

2.1 ms ± 8.47 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Just-in-time optimization with numba

In [20]:
from numba import jit

In [21]:
@jit
def my_nbop(data):
    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [22]:
my_nbop

CPUDispatcher(<function my_nbop at 0x7f04002ef5e0>)

In [23]:
%timeit my_nbop([1,2,3,4,5])

/common/software/install/migrated/anaconda/python3-2020.07-mamba/lib/python3.8/site-packages/numba/core/ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'data' of function 'my_nbop'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-21-de065de12a6b>", line 2:
@jit
def my_nbop(data):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


16.1 µs ± 8.82 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%timeit my_nbop(bigdata.copy())

781 µs ± 44.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
data2 = numpy.arange(1000000)

In [26]:
data2[0:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [27]:
%timeit my_nbop(data2)

1.52 ms ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Expression evaluators with numexpr

In [28]:
from numexpr import evaluate

In [29]:
smalldata = numpy.array([1,2,3,4,5])

In [30]:
evaluate("(smalldata**2 + 3) / 4.5")

array([0.88888889, 1.55555556, 2.66666667, 4.22222222, 6.22222222])

In [31]:
%timeit evaluate("(bigdata**2 + 3) / 4.5")

794 µs ± 30.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
